In [1]:
from langchain.agents import AgentType, initialize_agent
from langchain.llms import OpenAI
from langchain.agents.agent_toolkits.file_management import FileManagementToolkit

from dotenv import load_dotenv

import os

import sys
sys.path.append('..')
sys.path.append('../langchain_learnings/tools/slack/')
sys.path.append('../langchain_learnings/tools/jira/')

from langchain_learnings.tools.slack.slack_toolkit import SlackToolKit
from langchain_learnings.tools.slack.slack_api import SlackApiWrapper
from langchain_learnings.tools.jira.jira_api import JiraApiWrapper
from langchain_learnings.tools.jira.jira_toolkit import JiraToolkit

load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')
os.environ["JIRA_USERNAME"] = os.getenv('JIRA_USERNAME')
os.environ["JIRA_API_TOKEN"] = os.getenv('JIRA_API_TOKEN')
os.environ["JIRA_INSTANCE_URL"] = os.getenv('JIRA_INSTANCE_URL')
os.environ["SLACK_BOT_TOKEN"] = os.getenv('SLACK_BOT_TOKEN')

llm = OpenAI(temperature=0)

jira =  JiraApiWrapper()
jira_toolkit = JiraToolkit.from_jira_api_wrapper(jira)

slack = SlackApiWrapper()
slack_toolkit = SlackToolKit.from_slack_api_wrapper(slack)

tools = []
[tools.append(tool) for tool in jira_toolkit.get_tools()]
[tools.append(tool) for tool in slack_toolkit.get_tools()]

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True)

In [2]:
agent.run("make a new issue for project key OP to learn more python, post the jira id to #general in slack")



> Entering new AgentExecutor chain...


Creating issue "Learn more Python"


 I need to create a new issue in Jira and post the jira id to #general in Slack
Action: Create Issue
Action Input: {"summary": "Learn more Python", "description": "I want to learn more Python", "project": { "key": "OP" }, "issuetype": {"name": "Task"}, "priority": {"name": "Low"}}
Observation: {'id': '10007', 'key': 'OP-8', 'self': 'https://thinkovator.atlassian.net/rest/api/2/issue/10007'}
Thought: I now need to post the jira id to #general in Slack
Action: Write a chat message to the channel
Action Input: {"channel": "#general", "message": "The Jira ID for the new issue is OP-8"}
Observation: Successfully wrote chat message
Thought: I now know the final answer
Final Answer: The Jira ID for the new issue is OP-8

> Finished chain.


'The Jira ID for the new issue is OP-8'

In [ ]:
import requests
from requests.auth import HTTPBasicAuth
import json

base_url = os.getenv('JIRA_INSTANCE_URL')
create_url = f"{base_url}/rest/api/2/issue"

auth = HTTPBasicAuth(os.getenv('JIRA_USERNAME'), os.getenv('JIRA_API_TOKEN'))

metadata_url = f"{base_url}/rest/api/2/issue/createmeta"

headers = {
  "Accept": "application/json"
}

response = requests.request(
   "GET",
   metadata_url,
   headers=headers,
   auth=auth
)

print(json.dumps(json.loads(response.text), sort_keys=True, indent=4, separators=(",", ": ")))


In [ ]:

payload = json.dumps( {
  "fields": {
    "issuetype": {
      "name": "Task"
    },
    "priority": {
      "name": "Low"
    },
    "project": {
      "key": "OP"
    },
    "summary": "Main order flow broken",
    "description": "What do you think"
  }
} )

headers = {
  "Accept": "application/json",
  "Content-Type": "application/json"
}

response = requests.request(
   "POST",
   create_url,
   data=payload,
   headers=headers,
   auth=auth
)

print(response.status_code)
print(json.dumps(json.loads(response.text), sort_keys=True, indent=4, separators=(",", ": ")))